In [13]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [14]:
import numpy as np
import seaborn as sn; sn.set(font_scale=1.4)                     
import cv2                                 
import tensorflow as tf              
from PIL import Image
import urllib.request

In [15]:
class_names = ['pantalon', 'bicicleta', 'television', 'camiseta']
IMAGE_SIZE = (100, 100)

In [16]:
# Cargamos los datos de los ficheros y los mostramos

import json


with open('pantalones.json', 'r') as p:
        pantalones = json.load(p)
print(type(pantalones))
print(len(pantalones))


with open('bicicletas.json', 'r') as b:
        bicicletas = json.load(b)
print(type(bicicletas))
print(len(bicicletas))
with open('televisiones.json', 'r') as t:
        televisiones = json.load(t)
print(type(televisiones))
print(len(televisiones))

with open('camisetas.json', 'r') as c:
        camisetas = json.load(c)
print(type(camisetas))
print(len(camisetas))

<class 'list'>
40
<class 'list'>
40
<class 'list'>
40
<class 'list'>
40


In [17]:
# Hacemos un dataframe con los datos de los dos ficheros

import pandas as pd

tododatos = pantalones + bicicletas + televisiones + camisetas

df = pd.DataFrame(tododatos)
df


,id,title,description,distance,images,user,flags,visibility_flags,price,currency,free_shipping,web_slug,category_id,shipping,supports_shipping,shipping_allowed,seller_id,favorited,creation_date,modification_date,location,type_attributes,taxonomy,discount
0,wzy50mrwq5j5,Pantalón para de campana cuero marrón,pantalón de cuero de pata de campana de color ...,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'p8j359kvgyz9', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",18.00,EUR,False,pantalon-para-de-campana-cuero-marron-883281244,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,p8j359kvgyz9,False,1678873428724,1678873439187,"{'city': 'Albacete', 'postal_code': '02003', '...","{'brand': {'value': 'Bershka', 'title': 'Marca...",None,None
1,qzmywrkno1zv,pantalones,pantalones vaqueros de campana. Estan en muy b...,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '8z81807o1r63', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",10.00,EUR,False,pantalones-883111308,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,8z81807o1r63,False,1678809627731,1678809637929,"{'city': 'Albacete', 'postal_code': '02002', '...","{'size': {'value': '2', 'title': 'Talla', 'tex...",None,None
2,mzn8xyk91yzn,pantalón,talla 38 están nuevos me equivoque de talla,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'npj9k4v9kze7', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",5.00,EUR,False,pantalon-883104597,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,npj9k4v9kze7,False,1678808142425,1678808152827,"{'city': 'Albacete', 'postal_code': '02005', '...","{'gender': {'value': 'female', 'title': 'Géner...",None,None
3,e65glw2oypjo,pantalón,"nuevo en perfecto estado, talla M",56.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'p8j3e907m7z9', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",3.00,EUR,False,pantalon-883032248,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,p8j3e907m7z9,False,1678791766067,1678791776205,"{'city': 'Hellín', 'postal_code': '02400', 'co...","{'brand': {'value': 'Primark', 'title': 'Marca...",None,None
4,vjrr300ny4jk,Pantalon Levi's 511,"Pantalon Levi's original en perfecto estado, g...",1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '9nz0r8dre7zo', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",30.00,EUR,False,pantalon-levi-s-511-883158919,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,9nz0r8dre7zo,False,1678818895922,1678819425058,"{'city': 'Albacete', 'postal_code': '02005', '...","{'brand': {'value': 'Levi'S', 'title': 'Marca'...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0j23l0wmee6y,camiseta niña,Lote por 6. Talla 9 pero da una 7.\n,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'wzvyl1pyldzl', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",6.00,EUR,False,camiseta-nina-883215059,12461,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,wzvyl1pyldzl,False,1678832873903,1678832884052,"{'city': 'Albacete', 'postal_code': '02002', '...",None,None,None
156,wzy503ryxpj5,2 camisetas verano,"Son bastante cómodas, se amolda a el cuerpo y ...",2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '8z818lpemr63', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",17.00,EUR,False,2-camisetas-verano-88294312

In [18]:
# Eliminamos las columnas que no nos interesan
df_todosdatos = df.drop(['price', 'description'], axis=1)

df_todosdatos
# Aplicamos la función que hemos creado en el paso anterior a la columna title del dataframe
df_todosdatos['title'] = df_todosdatos.apply(lambda x: x['title'].lower(), axis=1)

df_todosdatos

,id,title,distance,images,user,flags,visibility_flags,currency,free_shipping,web_slug,category_id,shipping,supports_shipping,shipping_allowed,seller_id,favorited,creation_date,modification_date,location,type_attributes,taxonomy,discount
0,wzy50mrwq5j5,pantalón para de campana cuero marrón,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'p8j359kvgyz9', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalon-para-de-campana-cuero-marron-883281244,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,p8j359kvgyz9,False,1678873428724,1678873439187,"{'city': 'Albacete', 'postal_code': '02003', '...","{'brand': {'value': 'Bershka', 'title': 'Marca...",None,None
1,qzmywrkno1zv,pantalones,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '8z81807o1r63', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalones-883111308,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,8z81807o1r63,False,1678809627731,1678809637929,"{'city': 'Albacete', 'postal_code': '02002', '...","{'size': {'value': '2', 'title': 'Talla', 'tex...",None,None
2,mzn8xyk91yzn,pantalón,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'npj9k4v9kze7', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalon-883104597,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,npj9k4v9kze7,False,1678808142425,1678808152827,"{'city': 'Albacete', 'postal_code': '02005', '...","{'gender': {'value': 'female', 'title': 'Géner...",None,None
3,e65glw2oypjo,pantalón,56.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'p8j3e907m7z9', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalon-883032248,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,p8j3e907m7z9,False,1678791766067,1678791776205,"{'city': 'Hellín', 'postal_code': '02400', 'co...","{'brand': {'value': 'Primark', 'title': 'Marca...",None,None
4,vjrr300ny4jk,pantalon levi's 511,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '9nz0r8dre7zo', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,pantalon-levi-s-511-883158919,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,9nz0r8dre7zo,False,1678818895922,1678819425058,"{'city': 'Albacete', 'postal_code': '02005', '...","{'brand': {'value': 'Levi'S', 'title': 'Marca'...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,0j23l0wmee6y,camiseta niña,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': 'wzvyl1pyldzl', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,camiseta-nina-883215059,12461,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,wzvyl1pyldzl,False,1678832873903,1678832884052,"{'city': 'Albacete', 'postal_code': '02002', '...",None,None,None
156,wzy503ryxpj5,2 camisetas verano,2.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '8z818lpemr63', 'micro_name': '', 'imag...","{'pending': False, 'sold': False, 'reserved': ...","{'bumped': False, 'highlighted': False, 'urgen...",EUR,False,2-camisetas-verano-882943120,12465,"{'item_is_shippable': True, 'user_allows_shipp...",True,True,8z818lpemr63,False,1678742790336,1678742800509,"{'city': 'Albacete', 'postal_code': '02004', '...","{'brand': {'value': 'Bershka', 'title': 'Marca...",None,None
157,pj95l11lgvze,camiseta del valencia,1.0,[{'original': 'http://cdn.wallapop.com/images/...,"{'id': '0j2y4717rdzy', 'micro_name': '', 'imag...","{'pending

In [19]:
url = df_todosdatos['images'].iloc[0]
print(url)

[{'original': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800', 'xsmall': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W320', 'small': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W320', 'large': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800', 'medium': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W640', 'xlarge': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800', 'original_width': 0, 'original_height': 0}, {'original': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180295.jpg?pictureSize=W800', 'xsmall': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180295.jpg?pictureSize=W320', 'small': 'http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i31911

In [20]:
# Creamos una función que nos permita cargar las imágenes
def load_image(row):
  try:
    imagen = urllib.request.urlopen(row['images'])
    img_pil = Image.open(imagen)
    img_np = np.array(img_pil)
    img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    img_np = cv2.resize(img_np, IMAGE_SIZE)
    
    return img_np
  except: return None

In [21]:
imagen = urllib.request.urlopen("http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800http://cdn.wallapop.com/images/10420/el/vs/__/c10420p883281244/i3191180163.jpg?pictureSize=W800")
img_pil = Image.open(imagen)
img_np = np.array(img_pil)

print(img_np)

[[[179 185 185]
  [179 185 185]
  [179 185 185]
  ...
  [176 180 179]
  [176 180 179]
  [176 180 179]]

 [[179 185 185]
  [179 185 185]
  [179 185 185]
  ...
  [176 180 179]
  [176 180 179]
  [176 180 179]]

 [[179 185 185]
  [179 185 185]
  [180 186 186]
  ...
  [177 181 180]
  [176 180 179]
  [176 180 179]]

 ...

 [[146 136 127]
  [146 136 127]
  [146 136 127]
  ...
  [146 141 135]
  [146 141 135]
  [146 141 135]]

 [[146 136 127]
  [146 136 127]
  [146 136 127]
  ...
  [146 141 135]
  [146 141 135]
  [146 141 135]]

 [[144 134 125]
  [145 135 126]
  [144 134 125]
  ...
  [146 141 135]
  [146 141 135]
  [147 142 136]]]


In [22]:
# Aplicamos la función que hemos creado en el paso anterior a la columna images del dataframe para descargar todas las imágenes
df_todosdatos['images'] = df_todosdatos.apply(load_image, axis=1)


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
images = df_todosdatos['images']
title = df_todosdatos['title']

# Dividir los datos en conjuntos de entrenamiento y prueba
train_images, test_images, train_labels, test_labels = train_test_split(images, title, test_size=0.1, random_state=30)

# Mezclar los datos de entrenamiento
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

# Obtener el número de datos de entrenamiento y prueba
n_train = len(train_labels)
n_test = len(test_labels)

# Imprimir información
print("Número de datos de entrenamiento: {}".format(n_train))
print("Número de datos de prueba: {}".format(n_test))
print("Tamaño imagen: {}".format(IMAGE_SIZE))

Número de datos de entrenamiento: 144
Número de datos de prueba: 16
Cada imagen tiene un tamaño de: (100, 100)


In [24]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')


In [ ]:
from sklearn.model_selection import train_test_split
# Dividir los datos en conjuntos de entrenamiento y prueba
train_images, test_images, train_labels, test_labels = train_test_split(df_todosdatos['images'], df_todosdatos['category'], test_size=0.2, random_state=42)
train_images_tensor = []
for i in range(len(train_images)):
  train_images_tensor.append(tf.convert_to_tensor(train_images.iloc[i]))
np.array(train_images.tolist())

In [27]:
model.save('clf_imagenes.h5')